In [ ]:
! pip install git+https://github.com/m-bain/whisperx.git
# !pip install yt-dlp
!pip install git+https://github.com/m1guelpf/auto-subtitle.git
from google.colab import drive
drive.mount("/content/drive/")

# !yt-dlp https://www.youtube.com/watch?v=TBuIGBCF9jc -o /content/drive/MyDrive/data_bots/video
!whisperx /content/drive/MyDrive/data_bots/video.mp4 --model large-v2 --language en --align_model WAV2VEC2_ASR_LARGE_LV60K_960H --batch_size 4  --output_dir /content/drive/MyDrive/



  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-tm7yht4g
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-tm7yht4g
  Resolved https://github.com/m-bain/whisperx.git to commit 8227807fa9e076901ea4b4fbbf79c9777a6f5e03
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/SYSTRAN/faster-whisper.git (to revision 0.10.0) to /tmp/pip-install-g169v0ws/faster-whisper_56526b799158494fbbb7fbaed0605276
  Running command git clone --filter=blob:none --quiet https://github.com/SYSTRAN/faster-whisper.git /tmp/pip-install-g169v0ws/faster-whisper_56526b799158494fbbb7fbaed0605276
  Running command git checkout -q e1a218fab1ab02d637b79565995bf1a9c4c83a09
  Resolved https://github.com/SYSTRAN/faster-whisper.git to commit e1a218fab1ab02d637b79565995bf1a9c4c83a09
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/m1guelpf/auto-subtitle.git to /tmp/pip-req-build-51kl2cmk
  Running comman

In [ ]:
!whisperx --herlp

2024-01-15 12:28:52.333801: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 12:28:52.333844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 12:28:52.335779: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 12:28:54.259188: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set

In [ ]:
%ls ../

bin@                        datalab/  lib@     media/                    proc/        sbin@  tools/
boot/                       dev/      lib32@   mnt/                      python-apt/  srv/   usr/
content/                    etc/      lib64@   NGC-DL-CONTAINER-LICENSE  root/        sys/   var/
cuda-keyring_1.0-1_all.deb  home/     libx32@  opt/                      run/         tmp/


In [ ]:
!nvidia-smi

Mon Jan 15 12:29:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
def flatten_nested_list(data):
    flattened_data = []
    for item in data:
        if isinstance(item, list):
            flattened_data.extend(flatten_nested_list(item))
        else:
            flattened_data.append(item)
    return flattened_data


def split_text_into_lines(data):
    MaxChars = 30
    MaxDuration = 2.5
    MaxGap = 1.5

    subtitles = []
    line = []
    line_duration = 0
    line_chars = 0

    flattened_data = flatten_nested_list(data)

    for idx, word_data in enumerate(flattened_data):
        word = word_data.get("word", "")
        start = word_data.get("start", 0)
        end = word_data.get("end", 0)

        # Skip words without 'start' or 'end'
        if not start or not end:
            continue

        line.append(word_data)
        line_duration += end - start

        temp = " ".join(item.get("word", "") for item in line)

        new_line_chars = len(temp)
        duration_exceeded = line_duration > MaxDuration
        chars_exceeded = new_line_chars > MaxChars

        if idx > 0:
            gap = start - flattened_data[idx - 1].get('end', 0)
            maxgap_exceeded = gap > MaxGap
        else:
            maxgap_exceeded = False

        if duration_exceeded or chars_exceeded or maxgap_exceeded:
            if line:
                subtitle_line = {
                    "word": " ".join(item.get("word", "") for item in line),
                    "start": line[0].get("start", 0),
                    "end": line[-1].get("end", 0),
                    "textcontents": line
                }
                subtitles.append(subtitle_line)
                line = []
                line_duration = 0
                line_chars = 0

    if line:
        subtitle_line = {
            "word": " ".join(item.get("word", "") for item in line),
            "start": line[0].get("start", 0),
            "end": line[-1].get("end", 0),
            "textcontents": line
        }
        subtitles.append(subtitle_line)

    return subtitles

In [ ]:
import json

json_file_path = "/content/drive/MyDrive/video.json"

try:
    with open(json_file_path, 'r') as file:
        data = json.load(file)

except FileNotFoundError:
    print(f"File not found: {json_file_path}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")


In [ ]:

!pip install moviepy==2.0.0.dev2
!pip install imageio==2.25.1

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 100, in main
    with self.main_context():
  File "/usr/lib/python3.10/contextlib.py", line 142, in __exit__
    next(self.gen)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/command_context.py", line 19, in main_context
    with self._main_context:
  File "/usr/lib/python3.10/contextlib.py", line 576, in __exit__
    raise exc_details[1]
  File "/usr/lib/python3.10/contextlib.py", line 153, in __exit__
    self.gen.throw(typ, value, traceback)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/utils/temp_dir.py", line 70, in tempdir_registry
    yield _tempdir_registry
  File "/usr/lib/python3.10/contextl

In [ ]:
!apt --reinstall install imagemagick

Reading package lists... Done
^C


In [ ]:
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [ ]:
from moviepy.editor import TextClip, CompositeVideoClip, ColorClip, vfx
import numpy as np

def create_caption(textJSON, framesize, font="Helvetica-Bold", color='white', highlight_color='lightgreen', stroke_color='black', stroke_width=0.5):
    wordcount = len(textJSON['textcontents'])
    full_duration = textJSON['end'] - textJSON['start']

    word_clips = []
    xy_textclips_positions = []

    x_pos = 0
    y_pos = 0
    line_width = 0  # Total width of words in the current line
    frame_width = framesize[0]
    frame_height = framesize[1]

    x_buffer = frame_width * 1 / 10

    max_line_width = frame_width - 2 * (x_buffer)

    fontsize = int(frame_height * 0.075)  # 7.5 percent of video height

    space_width = ""
    space_height = ""

    for index, wordJSON in enumerate(textJSON['textcontents']):
        duration = wordJSON['end'] - wordJSON['start']
        word_clip = TextClip(wordJSON['word'], font=font, fontsize=fontsize, color=color, stroke_color=stroke_color,
                             stroke_width=stroke_width).set_start(textJSON['start']).set_duration(full_duration)
        word_clip_space = TextClip(" ", font=font, fontsize=fontsize, color=color).set_start(
            textJSON['start']).set_duration(full_duration)
        word_width, word_height = word_clip.size
        space_width, space_height = word_clip_space.size
        if line_width + word_width + space_width <= max_line_width:
            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos": x_pos,
                "y_pos": y_pos,
                "width": word_width,
                "height": word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos, y_pos))
            word_clip_space = word_clip_space.set_position((x_pos + word_width, y_pos))

            x_pos = x_pos + word_width + space_width
            line_width = line_width + word_width + space_width
        else:
            # Move to the next line
            x_pos = 0
            y_pos = y_pos + word_height + 10
            line_width = word_width + space_width

            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos": x_pos,
                "y_pos": y_pos,
                "width": word_width,
                "height": word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos, y_pos))
            word_clip_space = word_clip_space.set_position((x_pos + word_width, y_pos))
            x_pos = word_width + space_width

        word_clips.append(word_clip)
        word_clips.append(word_clip_space)

    for highlight_word in xy_textclips_positions:
        word_clip_highlight = TextClip(highlight_word['word'], font=font, fontsize=fontsize, color=highlight_color,
                                       stroke_color=stroke_color, stroke_width=stroke_width).set_start(
            highlight_word['start']).set_duration(highlight_word['duration'])


        word_clip_highlight = word_clip_highlight.set_position((highlight_word['x_pos'], highlight_word['y_pos']))
        word_clips.append(word_clip_highlight)

    return word_clips, xy_textclips_positions


In [ ]:
def find_highlight_segments(data, keyword):
    highlight_segments = []
    index = 0

    for segment in data['segments']:
        index+=1
        if keyword.lower() in segment['text'].lower():
          segment['index']=index
          highlight_segments.append(segment)

    return highlight_segments

keyword = 'everest'
highlight_segments = find_highlight_segments(data, keyword)

# Print the highlight segments
for segment in highlight_segments:
    print(f"Start: {segment['start']}, End: {segment['end']}, Text: {segment['text']}, Index: {segment['index']}")


In [ ]:
def find_segments_with_keyword(data, keyword):
    highlight_segments = []
    index = 0

    for segment in data['segments']:
        index+=1
        if keyword.lower() in segment['text'].lower():
          segment['index']=index
          highlight_segments.append(segment)

    return highlight_segments

def create_segments_list(segments, data, keyword, duration_limit=20):
    selected_segments = []

    index_for = 0

    for segment in segments:
        selected_chain = []

        total_duration = 0
        index_while = segment['index']+1
        index_while_temp = 0

        segment_duration = segment['end'] - segment['start']
        total_duration = segment_duration

        selected_chain.append(segment)


################### next segment not contain keyword  and keyword not in data[index_while+1] ###########
        while total_duration <= duration_limit and keyword not in data[index_while]["text"] :
            selected_chain.append(data[index_while])
            segment_duration = data[index_while]['end'] - data[index_while]['start']
            total_duration += segment_duration
            index_while_temp = index_while
            index_while = index_while_temp+1

        if total_duration >= duration_limit and total_duration <= duration_limit + duration_limit:
          selected_segments.append(selected_chain)


    return selected_segments

# Example

keyword_to_search = 'time'
keyword_segments = find_segments_with_keyword(data, keyword_to_search)

selected_segments = create_segments_list(keyword_segments, data['segments'],keyword_to_search)
# Output the selected segments
for segment in selected_segments:
    print(segment)


[{'start': 186.987, 'end': 194.511, 'text': " And I was talking to my fiance today about, it's kind of strange how sometimes I have a conversation between David Goggins and Goggins.", 'words': [{'word': 'And', 'start': 186.987, 'end': 187.087, 'score': 0.806}, {'word': 'I', 'start': 187.167, 'end': 187.207, 'score': 0.996}, {'word': 'was', 'start': 187.247, 'end': 187.367, 'score': 0.87}, {'word': 'talking', 'start': 187.387, 'end': 187.628, 'score': 0.874}, {'word': 'to', 'start': 187.668, 'end': 187.728, 'score': 0.994}, {'word': 'my', 'start': 187.768, 'end': 187.888, 'score': 0.843}, {'word': 'fiance', 'start': 187.928, 'end': 188.348, 'score': 0.919}, {'word': 'today', 'start': 188.388, 'end': 188.648, 'score': 0.833}, {'word': 'about,', 'start': 188.688, 'end': 188.948, 'score': 0.884}, {'word': "it's", 'start': 189.409, 'end': 189.529, 'score': 0.8}, {'word': 'kind', 'start': 189.549, 'end': 189.689, 'score': 0.879}, {'word': 'of', 'start': 189.709, 'end': 189.749, 'score': 0.91

In [ ]:
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords


import nltk
nltk.download('stopwords')

def highlights_keywords(data):
    stop_words = set(stopwords.words('french'))  **

    all_words = []

    for segment in data['segments']:
        text = segment['text'].lower()
        words = text.split()
        words = [word for word in words if word.isalpha() and word not in stop_words]
        all_words.extend(words)

    word_counts = Counter(all_words)


    word_count_df = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Count'])
    word_count_df = word_count_df.sort_values(by='Count', ascending=False).reset_index(drop=True)

    return word_count_df


word_count_df = highlights_keywords(data)
word_count_df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Word,Count
0,to,462
1,i,440
2,and,439
3,the,397
4,you,388
...,...,...
1238,form,1
1239,interactions,1
1240,lack,1
1241,diet,1


In [ ]:

from moviepy.editor import TextClip, CompositeVideoClip, concatenate_videoclips,VideoFileClip, ColorClip, clips_array


def adjust_timestamps(subtitles):



    global_start = min(segment['start'] for segment in subtitles)

    print(global_start)
    for segment in subtitles:
        segment_start = segment['start']
        segment_end = segment['end']


        segment['start'] -= global_start
        segment['end'] -= global_start

        for word_info in segment['textcontents']:
            word_info['start'] -= global_start
            word_info['end'] -= global_start


    return subtitles

def auto_subtitles(input_path,subtitles, output_video_path,start_time,end_time):
  clip = VideoFileClip(input_path).subclip(start_time, end_time)
  second_clip = VideoFileClip("/content/drive/MyDrive/data_bots/minecraft.mp4").subclip(start_time, end_time)
  resized_clip = crop(clip,x_center=960, y_center=540, width=607, height=540)
  resized_second_clip = crop(second_clip,x_center=960, y_center=540, width=607, height=540)

  input_video_second = resized_second_clip
  input_video = resized_clip
  frame_size = input_video.size
  frame_size_second = input_video_second.size


  all_linelevel_splits=[]

  for line in subtitles:
    out_clips,positions = create_caption(line,frame_size_second)

    max_width = 0
    max_height = 0

    for position in positions:
      # print (out_clip.pos)
      # break
      x_pos, y_pos = position['x_pos'],position['y_pos']
      width, height = position['width'],position['height']

      max_width = max(max_width, x_pos + width)
      max_height = max(max_height, y_pos + height)

    color_clip = ColorClip(size=(int(max_width*1.1), int(max_height*1.1)),
                        color=(64, 64, 64))
    color_clip = color_clip.set_opacity(.6)
    color_clip = color_clip.set_start(line['start']).set_duration(line['end']-line['start'])

    # centered_clips = [each.set_position('center') for each in out_clips]

    clip_to_overlay = CompositeVideoClip([color_clip]+ out_clips)
    clip_to_overlay = clip_to_overlay.set_position("bottom")


    all_linelevel_splits.append(clip_to_overlay)



  final_video = CompositeVideoClip([input_video] + all_linelevel_splits)

  # Set the audio of the final video to be the same as the input video
  final_video = final_video.set_audio(input_video.audio)
  combined = clips_array([[final_video],
                          [input_video_second]])
  print("input_video_duration : ",input_video_second.duration,"//", "final_video_duration : ", final_video.duration)


  combined.write_videofile(output_video_path, fps=24, codec="libx264", audio_codec="aac")
  final_video.close()
  combined.close()
  clip.close()
  input_video.close()
  resized_clip.close()



In [ ]:

from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.fx.all import resize, crop
import subprocess
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
from moviepy.editor import TextClip, CompositeVideoClip, concatenate_videoclips,VideoFileClip, ColorClip, vfx

# !pip install ffmpeg-python

def cut_video(input_path, output_path, start_time, end_time):
    clip = VideoFileClip(input_path).subclip(start_time, end_time)
    clip2 = VideoFileClip()
    print("size",clip.size)
    resized_clip = crop(clip,x_center=960, y_center=270, width=607, height=540)



    return resized_clip

directory = "/content/drive/MyDrive/data_bots/"
directory_ = "/content/drive/MyDrive/data_bots/subtitled/"
input_video_path = directory + "video.mp4"
subtitles_output_directory = directory + "subtitled/"
subtitles_all_video=[]

for segment in selected_segments:
  output_video_path = directory + "video" + str(segment[0]["start"])+ "_" + str(segment[-1]["end"])+".mp4"
  output_video_path__ = directory_ + "video" + str(segment[0]["start"])+ "_" + str(segment[-1]["end"])+".mp4"
  output_video_path_ = "../" + "video" + str(segment[0]["start"])+ "_" + str(segment[-1]["end"])+".mp4"
  words_timestamps = []


  for seg in segment:
    words_timestamps.append(seg["words"])

  subtitles = split_text_into_lines(words_timestamps)
  subtitles_adjusted = adjust_timestamps(subtitles)

  # resized_clip = cut_video(input_video_path,output_video_path,segment[0]["start"], segment[-1]["end"] )
  auto_subtitles(input_video_path,subtitles_adjusted, output_video_path__,segment[0]["start"], segment[-1]["end"])

  # !auto_subtitle {output_video_path} -o {subtitles_output_directory} --task translate --model medium


186.987
input_video_duration :  27.375 // final_video_duration :  27.375
Moviepy - Building video /content/drive/MyDrive/data_bots/subtitled/video186.987_214.362.mp4.
MoviePy - Writing audio in video186.987_214.362TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/data_bots/subtitled/video186.987_214.362.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data_bots/subtitled/video186.987_214.362.mp4
273.519
input_video_duration :  32.82600000000002 // final_video_duration :  32.82600000000002
Moviepy - Building video /content/drive/MyDrive/data_bots/subtitled/video273.519_306.345.mp4.
MoviePy - Writing audio in video273.519_306.345TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/data_bots/subtitled/video273.519_306.345.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data_bots/subtitled/video273.519_306.345.mp4
15.728999999999985
input_video_duration :  27.805000000000007 // final_video_duration :  266.993
Moviepy - Building video /content/drive/MyDrive/data_bots/subtitled/video278.54_306.345.mp4.
MoviePy - Writing audio in video278.54_306.345TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data_bots/subtitled/video708.099_751.838.mp4
830.869
input_video_duration :  28.644999999999982 // final_video_duration :  28.644999999999982
Moviepy - Building video /content/drive/MyDrive/data_bots/subtitled/video830.869_859.514.mp4.
MoviePy - Writing audio in video830.869_859.514TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/data_bots/subtitled/video830.869_859.514.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data_bots/subtitled/video830.869_859.514.mp4
865.683
input_video_duration :  39.777000000000044 // final_video_duration :  39.777000000000044
Moviepy - Building video /content/drive/MyDrive/data_bots/subtitled/video865.683_905.46.mp4.
MoviePy - Writing audio in video865.683_905.46TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/data_bots/subtitled/video865.683_905.46.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data_bots/subtitled/video865.683_905.46.mp4
924.222
